# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Li  ->  J. Li  |  ['J. Li']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 72 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.20170


extracting tarball to tmp_2508.20170...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.20229


extracting tarball to tmp_2508.20229...

...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{2020A&A...641A...6P}{Planck Collaboration}, \emph{{Planck 2018 results. VI. Cosmological  parameters}}, \href{https://doi.org/10.1051/0004-6361/201833910}{\emph{\aap}  {\bfseries 641} (2020) A6}  [\href{https://arxiv.org/abs/1807.06209}{{\ttfamily 1807.06209}}].
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3}).*?href(.*?{(?P<url>http[\S]*)})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{2019A&A...631A..40S}S.~{Schuldt}, G.~{Chiriv{ì}}, S.~H. {Suyu}, A.~{Y{\i}ld{\i}r{\i}m},  A.~{Sonnenfeld}, A.~{Halkola} et~al., \emph{{Inner dark matter distribution  of the Cosmic Horseshoe (J1148+1930) with gravitational lens

extracting tarball to tmp_2508.20860...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2508.20860/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors/authorlist' from 'tmp_2508.20860/authors/authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 79 bibliographic references in tmp_2508.20860/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.20229-b31b1b.svg)](https://arxiv.org/abs/2508.20229) | **Combined dark matter search towards dwarf spheroidal galaxies with Fermi-LAT, HAWC, H.E.S.S., MAGIC, and VERITAS**  |
|| F.-L. Collaboration, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-08-29*|
|*Comments*| **|
|**Abstract**|            Dwarf spheroidal galaxies (dSphs) are excellent targets for indirect dark matter (DM) searches using gamma-ray telescopes because they are thought to have high DM content and a low astrophysical background. The sensitivity of these searches is improved by combining the observations of dSphs made by different gamma-ray telescopes. We present the results of a combined search by the most sensitive currently operating gamma-ray telescopes, namely: the satellite-borne Fermi-LAT telescope; the ground-based imaging atmospheric Cherenkov telescope arrays H.E.S.S., MAGIC, and VERITAS; and the HAWC water Cherenkov detector. Individual datasets were analyzed using a common statistical approach. Results were subsequently combined via a global joint likelihood analysis. We obtain constraints on the velocity-weighted cross section $\langle \sigma \mathit{v} \rangle$ for DM self-annihilation as a function of the DM particle mass. This five-instrument combination allows the derivation of up to 2-3 times more constraining upper limits on $\langle \sigma \mathit{v} \rangle$ than the individual results over a wide mass range spanning from 5 GeV to 100 TeV. Depending on the DM content modeling, the 95% confidence level observed limits reach $1.5\times$10$^{-24}$ cm$^3$s$^{-1}$ and $3.2\times$10$^{-25}$ cm$^3$s$^{-1}$, respectively, in the $\tau^+\tau^-$ annihilation channel for a DM mass of 2 TeV.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.20860-b31b1b.svg)](https://arxiv.org/abs/2508.20860) | **Euclid preparation. LXXIV. Euclidised observations of Hubble Frontier Fields and CLASH galaxy clusters**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-08-29*|
|*Comments*| *18 pages, 12 figures, 1 table, A&A in press*|
|**Abstract**|            We present HST2EUCLID, a novel Python code to generate Euclid realistic mock images in the $H_{\rm E}$, $J_{\rm E}$, $Y_{\rm E}$, and $I_{\rm E}$ photometric bands based on panchromatic Hubble Space Telescope observations. The software was used to create a simulated database of Euclid images for the 27 galaxy clusters observed during the Cluster Lensing And Supernova survey with Hubble (CLASH) and the Hubble Frontier Fields (HFF) program. Since the mock images were generated from real observations, they incorporate, by construction, all the complexity of the observed galaxy clusters. The simulated Euclid data of the galaxy cluster MACS J0416.1$-$2403 were then used to explore the possibility of developing strong lensing models based on the Euclid data. In this context, complementary photometric or spectroscopic follow-up campaigns are required to measure the redshifts of multiple images and cluster member galaxies. By Euclidising six parallel blank fields obtained during the HFF program, we provide an estimate of the number of galaxies detectable in Euclid images per ${\rm deg}^2$ per magnitude bin (number counts) and the distribution of the galaxy sizes. Finally, we present a preview of the Chandra Deep Field South that will be observed during the Euclid Deep Survey and two examples of galaxy-scale strong lensing systems residing in regions of the sky covered by the Euclid Wide Survey. The methodology developed in this work lends itself to several additional applications, as simulated Euclid fields based on HST (or JWST) imaging with extensive spectroscopic information can be used to validate the feasibility of legacy science cases or to train deep learning techniques in advance, thus preparing for a timely exploitation of the Euclid Survey data.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.20170-b31b1b.svg)](https://arxiv.org/abs/2508.20170) | **Optical Spectroscopy of the Most Compact Accreting Binary Harboring a Magnetic White Dwarf and a Hydrogen-rich Donor**  |
|| I. Galiullin, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-08-29*|
|*Comments*| *11 pages, 6 figures, 1 table. Accepted for publication in ApJ Letters*|
|**Abstract**|            Accreting white dwarfs in close binary systems, commonly known as cataclysmic variables (CVs), with orbital periods below the canonical period minimum ($\approx$ 80 minutes) are rare. Such short periods can only be reached if the donor star in the CV is either significantly evolved before initiating mass transfer to the white dwarf (WD) or metal-poor. We present optical photometry and spectroscopy of Gaia19bxc, a high-amplitude variable identified as a polar CV with an exceptionally short orbital period of 64.42 minutes - well below the canonical CV period minimum. High-speed photometry confirms persistent double-peaked variability consistent with cyclotron beaming, thus indicating the presence of a magnetic WD. Phase-resolved Keck/LRIS spectroscopy reveals strong hydrogen and helium emission lines but no donor features, indicating the accretor is a magnetic WD and the donor is hydrogen-rich, but cold and faint. The absence of a detectable donor and the low inferred temperature ($\lesssim$ 3500 K) disfavor an evolved donor scenario. Instead, the short period and the system's halo-like kinematics suggest Gaia19bxc may be the first known metal-poor polar. Because metal-poor donors are more compact than solar-metallicity donors of the same mass, they can reach shorter minimum periods. Gaia19bxc is one of only a handful of known metal-poor CVs below the canonical period minimum and has the shortest period of any such magnetic system discovered to date.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.20229/./plots/appendix/BootesI.png', 'tmp_2508.20229/./plots/appendix/CanesVenaticiI.png', 'tmp_2508.20229/./plots/appendix/CanesVenaticiII.png', 'tmp_2508.20229/./plots/appendix/Carina.png', 'tmp_2508.20229/./plots/appendix/ComaBerenices.png', 'tmp_2508.20229/./plots/appendix/Draco.png', 'tmp_2508.20229/./plots/appendix/Fornax.png', 'tmp_2508.20229/./plots/appendix/Hercules.png', 'tmp_2508.20229/./plots/appendix/LeoI.png', 'tmp_2508.20229/./plots/appendix/LeoII.png', 'tmp_2508.20229/./plots/appendix/LeoIV.png', 'tmp_2508.20229/./plots/appendix/LeoV.png', 'tmp_2508.20229/./plots/appendix/LeoT.png', 'tmp_2508.20229/./plots/appendix/Sculptor.png', 'tmp_2508.20229/./plots/appendix/Segue1.png', 'tmp_2508.20229/./plots/appendix/Segue2.png', 'tmp_2508.20229/./plots/appendix/Sextans.png', 'tmp_2508.20229/./plots/appendix/UrsaMajorI.png', 'tmp_2508.20229/./plots/appendix/UrsaMajorII.png', 'tmp_2508.20229/./plots/appendix/UrsaMinor.png', 'tmp_2508.20229/./plots/limits/A

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sv}{\langle \sigma \mathit{v} \rangle}$
$\newcommand$</div>



<div id="title">

# $\boldmath$ Combined dark matter search towards dwarf spheroidal galaxies with _Fermi_-LAT, HAWC, H.E.S.S., MAGIC, and VERITAS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.20229-b31b1b.svg)](https://arxiv.org/abs/2508.20229)<mark>Appeared on: 2025-08-29</mark> - 

</div>
<div id="authors">

F.-L. Collaboration, et al. -- incl., <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** Dwarf spheroidal galaxies (dSphs) are excellent targets for indirect dark matter (DM) searches using gamma-ray telescopes because they are thought to have high DM content and a low astrophysical background. The sensitivity of these searches is improved by combining the observations of dSphs made by different gamma-ray telescopes. We present the results of a combined search by the most sensitive currently operating gamma-ray telescopes, namely: the satellite-borne _Fermi_ -LAT telescope; the ground-based imaging atmospheric Cherenkov telescope arrays H.E.S.S., MAGIC, and VERITAS; and the HAWC water Cherenkov detector. Individual datasets were analyzed using a common statistical approach. Results were subsequently combined via a global joint likelihood analysis. We obtain constraints on the velocity-weighted cross section $\sv$ for DM self-annihilation as a function of the DM particle mass. This five-instrument combination allows the derivation of up to 2-3 times more constraining upper limits on $\sv$ than the individual results over a wide mass range spanning from 5 GeV to 100 TeV. Depending on the DM content modeling, the 95 \% confidence level observed limits reach $1.5\times$ 10 $^{-24}$ cm $^3$ s $^{-1}$ and $3.2\times$ 10 $^{-25}$ cm $^3$ s $^{-1}$ , respectively, in the $\tau^+\tau^-$ annihilation channel for a DM mass of 2 TeV.

</div>

<div id="div_fig1">

<img src="tmp_2508.20229/./plots/appendix/BootesI.png" alt="Fig5.1" width="10%"/><img src="tmp_2508.20229/./plots/appendix/CanesVenaticiI.png" alt="Fig5.2" width="10%"/><img src="tmp_2508.20229/./plots/appendix/CanesVenaticiII.png" alt="Fig5.3" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Carina.png" alt="Fig5.4" width="10%"/><img src="tmp_2508.20229/./plots/appendix/ComaBerenices.png" alt="Fig5.5" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Draco.png" alt="Fig5.6" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Fornax.png" alt="Fig5.7" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Hercules.png" alt="Fig5.8" width="10%"/><img src="tmp_2508.20229/./plots/appendix/LeoI.png" alt="Fig5.9" width="10%"/><img src="tmp_2508.20229/./plots/appendix/LeoII.png" alt="Fig5.10" width="10%"/>

**Figure 5. -** Comparisons between the $J$-factors used for this study versus the angular radius. Computed $J$-factors from 2015ApJ...801...74G($\mathcal{GS}$ set in Table \ref{tab:j-factor}) are extended up to the outermost visible star and are in blue. The $J$-factors computed from 2015MNRAS.446.3002B, 2015MNRAS.453..849B($\mathcal{B}$ set in Table \ref{tab:j-factor}) are extended up to the tidal radius of the dwarfs and are in orange. The solid lines represent the most probable value of the $J$-factors while the shaded regions correspond to the 1$\sigma$ standard deviation. (*fig:comparison-j-factors-1*)

</div>
<div id="div_fig2">

<img src="tmp_2508.20229/./plots/appendix/LeoIV.png" alt="Fig6.1" width="10%"/><img src="tmp_2508.20229/./plots/appendix/LeoV.png" alt="Fig6.2" width="10%"/><img src="tmp_2508.20229/./plots/appendix/LeoT.png" alt="Fig6.3" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Sculptor.png" alt="Fig6.4" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Segue1.png" alt="Fig6.5" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Segue2.png" alt="Fig6.6" width="10%"/><img src="tmp_2508.20229/./plots/appendix/Sextans.png" alt="Fig6.7" width="10%"/><img src="tmp_2508.20229/./plots/appendix/UrsaMajorI.png" alt="Fig6.8" width="10%"/><img src="tmp_2508.20229/./plots/appendix/UrsaMajorII.png" alt="Fig6.9" width="10%"/><img src="tmp_2508.20229/./plots/appendix/UrsaMinor.png" alt="Fig6.10" width="10%"/>

**Figure 6. -** Continuation of Fig. \ref{fig:comparison-j-factors-1}. (*fig:comparison-j-factors-2*)

</div>
<div id="div_fig3">

<img src="tmp_2508.20229/./plots/limits/Annihilation_bb_Geringer-Sameth_Combination_bands.png" alt="Fig2.1" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_tt_Geringer-Sameth_Combination_bands.png" alt="Fig2.2" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_WW_Geringer-Sameth_Combination_bands.png" alt="Fig2.3" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_ZZ_Geringer-Sameth_Combination_bands.png" alt="Fig2.4" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_ee_Geringer-Sameth_Combination_bands.png" alt="Fig2.5" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_mumu_Geringer-Sameth_Combination_bands.png" alt="Fig2.6" width="14%"/><img src="tmp_2508.20229/./plots/limits/Annihilation_tautau_Geringer-Sameth_Combination_bands.png" alt="Fig2.7" width="14%"/>

**Figure 2. -** Upper limits at 95\% confidence level on $\sv$ as a function of the DM mass for seven annihilation channels, using the set of $J$-factors from 2015ApJ...801...74G($\mathcal{GS}$ set in Table \ref{tab:j-factor}). The black solid line represents the observed combined limits obtained for the 20 dSphs included in this work, the blue dashed line is the median of the null hypothesis ($H_0$) corresponding to the expected limits with no DM signal ($\langle \sigma v \rangle = 0 $), while the green and yellow bands show the 68\% and 95\% containment bands. Upper limits for each individual instrument are also indicated. The value of the thermal relic cross section as a function of the DM mass is given as the gray dotted-dashed line 2012PhRvD..86b3506S. (*fig:limits-geringer-sameth*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.20229"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\LEt}[1]{\textcolor{red}{LE: #1}}$
$\newcommand{\T}{Table }$
$\newcommand{\Sec}{Sect. }$
$\newcommand{\Fig}{Fig. }$
$\newcommand{\Eq}{Eq. }$
$\newcommand{\HS}{\texttt{HST2EUCLID}}$
$\newcommand{\arraystretch}{1.6}$</div>



<div id="title">

# $\Euclid$ preparation: LXXIV. Euclidised observations of Hubble Frontier Fields and CLASH galaxy clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.20860-b31b1b.svg)](https://arxiv.org/abs/2508.20860)<mark>Appeared on: 2025-08-29</mark> -  _18 pages, 12 figures, 1 table, A&A in press_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present $\HS$ , a novel \texttt{Python} code to generate $\Euclid$ realistic mock images in the $\HE$ , $\JE$ , $\YE$ , and $\IE$ photometric bands based on panchromatic $_ Hubble_$ Space Telescope observations.    The software was used to create a simulated database of $\Euclid$ images for the 27 galaxy clusters observed during the Cluster Lensing And Supernova survey with $_ Hubble_$ (CLASH) and the Hubble Frontier Fields (HFF) program. Since the mock images were generated from real observations, they incorporate, by construction, all the complexity of the observed galaxy clusters.    The simulated $\Euclid$ data of the galaxy cluster MACS J0416.1 $-$ 2403 were then used to explore the possibility of developing strong lensing models based on the $\Euclid$ data. In this context, complementary photometric or spectroscopic follow-up campaigns are required to measure the redshifts of multiple images and cluster member galaxies. By Euclidising six parallel blank fields obtained during the HFF program, we provide an estimate of the number of galaxies detectable in $\Euclid$ images per ${\rm deg}^2$ per magnitude bin (number counts) and the distribution of the galaxy sizes. Finally, we present a preview of the $_ Chandra_$ Deep Field South that will be observed during the Euclid Deep Survey and two examples of galaxy-scale strong lensing systems residing in regions of the sky covered by the Euclid Wide Survey. The methodology developed in this work lends itself to several additional applications, as simulated $\Euclid$ fields based on HST (or JWST) imaging with extensive spectroscopic information can be used to validate the feasibility of legacy science cases or to train deep learning techniques in advance, thus preparing for a timely exploitation of the Euclid Survey data.

</div>

<div id="div_fig1">

<img src="tmp_2508.20860/./figures/PIPELINE.png" alt="Fig1" width="100%"/>

**Figure 1. -** Steps of the Euclidisation pipeline. In the top panel (a), we show the original _ Hubble_ F606W image of the galaxy cluster MACS J0416 at $z=0.396$. The bottom panel (e) shows the final simulated \IE band. The intermediate panels (b), (c), and (d), from top to bottom, illustrate: the convolution to match the \Euclid PSF, the result of combining the _ Hubble_ F606W and F814W filters to produce the \IE band, and the re-binning necessary to match the \Euclid pixel scale in the \IE band (see $\Sec$\ref{sec:pipeline}). (*fig:pipeline*)

</div>
<div id="div_fig2">

<img src="tmp_2508.20860/./figures/filters-paper.png" alt="Fig5" width="100%"/>

**Figure 5. -** Transmission curves of HST and \Euclid filters considered in this work. The five HST filters F606W, F814W, F105W, F125W, and F160W are shown with coloured solid lines, while the four \Euclid bands \IE, \YE, \JE, and \HE are represented by the coloured areas. The vertical black dashed lines indicate the wavelengths where the response of the HST F814W, F125W, and F160W filters begins to dominate over the F606W, F105W, and F125W filters, respectively. (*fig:filters*)

</div>
<div id="div_fig3">

<img src="tmp_2508.20860/./figures/mass_profile.png" alt="Fig4" width="100%"/>

**Figure 4. -** *Top panel*: Cumulative total mass profiles of MACS J0416 as derived from the B23, $\mathrm{EM_{spec}}$, and $\mathrm{EM_{phot}}$ strong lensing models. We show the results for the three models using dark grey, green, and red colours, respectively. The coloured bands indicate the 68.3\% confidence intervals. We also show the cumulative total mass profiles for the cluster member component (CM) using lighter colours. The radial distances are measured with respect to the BCG-N. We indicate the positions of multiple images and cluster members used in the \Euclid-based and B23 models with red and black vertical sticks at the top and bottom of the figure, respectively. *Bottom panel*: Ratios between the cumulative total mass profiles derived from the \Euclid-based and the B23 models. The coloured bands indicate the 68.3\% confidence intervals. (*fig:mass_profile*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.20860"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

127  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
